In [1]:
import pandas as pd
import numpy as np

In [2]:
# PC
data = pd.read_csv(r'C:\Users\Ezequiel\Desktop\machine-learning-2c-2025\data.csv', low_memory=False, sep='|')
# Notebook
#data = pd.read_csv(r'C:\Users\forma\Desktop\machine-learning-2c-2025\data.csv', low_memory=False, sep='|')

In [3]:
data.columns

Index(['client_id', 'Target', 'Month', 'First_product_dt', 'Last_product_dt',
       'CreditCard_Premium', 'CreditCard_Active', 'CreditCard_CoBranding',
       'Loan_Active', 'Mortgage_Active', 'SavingAccount_Active_ARG_Salary',
       'SavingAccount_Active_ARG', 'SavingAccount_Active_DOLLAR',
       'DebitCard_Active', 'Investment_Active', 'Package_Active',
       'Insurance_Life', 'Insurance_Home', 'Insurance_Accidents',
       'Insurance_Mobile', 'Insurance_ATM', 'Insurance_Unemployment', 'Sex',
       'Client_Age_grp', 'SavingAccount_Balance_FirstDate',
       'SavingAccount_Balance_LastDate', 'SavingAccount_Balance_Average',
       'SavingAccount_Days_with_use', 'SavingAccount_Days_with_Credits',
       'SavingAccount_Days_with_Debits',
       'SavingAccount_Salary_Payment_Transactions',
       'SavingAccount_Transfer_In_Transactions',
       'SavingAccount_ATM_Extraction_Transactions',
       'SavingAccount_Service_Payment_Transactions',
       'SavingAccount_CreditCard_Payment_T

In [4]:
print(data['client_id'].value_counts())

client_id
4221845                   9
7219504                   9
5882370                   9
1158013                   9
3914580                   9
                         ..
6623284                   1
672542                    1
264018                    1
4424661                   1
(238615 rows affected)    1
Name: count, Length: 26561, dtype: int64


In [5]:
# Busco los ID´s de los clientes que tienen 9 meses de data

# Creo una tabla con ID_cliente, cantidad de apariciones
ids_clientes_9m = data['client_id'].value_counts()

# Me quedo solo con aquellos que aparecen 9 veces 
ids_clientes_9m = ids_clientes_9m[ids_clientes_9m == 9].index

# Me quedo con todos los datos de quienes aparecen 9 meses
data_clientes_9m = data[data['client_id'].isin(ids_clientes_9m)]

data_clientes_9m.shape

(238347, 77)

In [6]:
# [agosto, septiembre, octubre, noviembre, diciembre, enero] TW, [febrero] LW, [marzo, abril] PW

# me quedo con los clientes que en el ultimo mes de la TW tienen package_active = no & CreditCard_CoBranding = no, y ademas, aparecen en los 9 meses
filtered_bussiness_conditions = data_clientes_9m[(data_clientes_9m['Month'] == '2019-01-01') 
                    & (data_clientes_9m['Package_Active'] == 'No')
                    & (data_clientes_9m['CreditCard_CoBranding'] == 'No')]

# Obtengo sus id
ids_filtered_bussiness_conditions = filtered_bussiness_conditions['client_id'].values
ids_filtered_bussiness_conditions.shape

(23191,)

In [7]:
#se toman los ultimos 2 meses
data_TGT = data[data['Month'].isin(['2019-03-01', '2019-04-01'])][['client_id', 'Target']]

#se buscan los que esten dentro del target y despues se mergea con el general quedando los ids con y sin target
data_TGT = data[(data['Month'].isin(['2019-03-01', '2019-04-01'])) 
       & (data['Target'] == 1)][['client_id']].drop_duplicates()

data_TGT['TGT'] = 1

data_final_simple = pd.merge(filtered_bussiness_conditions['client_id'].drop_duplicates(),data_TGT,how='left',on='client_id')

data_final_simple['TGT'] = data_final_simple['TGT'].fillna(0)

data_final_simple['TGT'].value_counts()

TGT
0.0    16368
1.0     6823
Name: count, dtype: int64

In [8]:
# Ahora unifico la tabla de clientes filtrados por paquete con los que aparecen 9 meses 
data_final = data[data['client_id'].isin(ids_filtered_bussiness_conditions)]
data_final.shape

(208719, 77)

In [9]:
# Ahora ya puedo separar las ventanas
training_window = data_final[data_final['Month'].isin(['2018-08-01',    
'2018-09-01',
'2018-10-01',
'2018-11-01',
'2018-12-01',
'2019-01-01'   
])]

training_window.shape

(139146, 77)

In [10]:
lead_window = data_final[data_final['Month'].isin(['2019-02-01'])]
lead_window.shape

(23191, 77)

In [11]:
training_window['Target'].value_counts()

Target
0.0    98208
1.0    40938
Name: count, dtype: int64

In [12]:
# Ahora tengo que empezar a limpiar los datos, empiezo por ver las columnas que contengan nulos
training_window.columns[training_window.isnull().any()].tolist()

['SavingAccount_Balance_Average', 'Region', 'CreditCard_Product']

In [13]:
# 3 columnas tienen nulos

# 1) SavingAccount_Balance_Average: Esta es una de las columnas que presenta nulos, es el promedio del monto que se tiene en la cuenta de ahorros
print(training_window['SavingAccount_Balance_Average'])

0          406.35
1           73.56
2            0.00
3         2366.02
4            0.40
           ...   
238597       0.00
238603       0.00
238604     367.43
238608       0.00
238612        NaN
Name: SavingAccount_Balance_Average, Length: 139146, dtype: float64


In [14]:
# Reemplazo los valores nulos con Balance_FirstDate - Debits + Credits
training_window.loc[:, 'SavingAccount_Balance_Average'] = (
    training_window['SavingAccount_Balance_Average'].fillna(
        training_window['SavingAccount_Balance_FirstDate']
        - training_window['SavingAccount_Debits_Amounts']
        + training_window['SavingAccount_Credits_Amounts']
    )
)

In [15]:
# 2) Region: Es la segunda columna con nulos 
print(training_window['Region'])

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
         ... 
238597    NaN
238603    NaN
238604    NaN
238608    NaN
238612    NaN
Name: Region, Length: 139146, dtype: object


In [16]:
# Parecen todos nulos, me fijo si tiene valores diferentes
training_window['Region'].value_counts()

Series([], Name: count, dtype: int64)

In [17]:
# Como son todos nulos, voy a buscar el dato de region mas cercano en el tiempo de cada cliente  y voy a suponer que 
# no cambio 
region_data = data_final.loc[data_final['Month'].isin(['2019-03-01']), ['client_id', 'Region']]
region_data

,client_id,Region
35,2739521,AMBA Resto
50,5025967,REGION PATAGONICA
82,2799736,REGION PATAGONICA
94,6722691,REGION CENTRO
101,5425606,BUENOS AIRES
...,...,...
238606,7328811,REGION NORTE GRANDE ARGENTINO
238607,6687431,REGION NORTE GRANDE ARGENTINO
238609,1899373,REGION CENTRO
238610,6448833,REGION NORTE GRANDE ARGENTINO


In [18]:
# PArece todo bien, pero me fijo si quedo algun nulo
region_data['Region'].isna().sum()

np.int64(6)

In [19]:
# Quedan 6 valores nulos, los relleno con la region mas popular
region_data['Region'].value_counts()

Region
BUENOS AIRES                     6984
REGION CENTRO                    4579
REGION NORTE GRANDE ARGENTINO    3674
REGION PATAGONICA                2432
CABA Centro/Norte                2048
AMBA Resto                       1828
REGION CUYO                      1640
Name: count, dtype: int64

In [20]:
region_data[region_data['Region'].isna()]

,client_id,Region
48964,436555,NaN
100092,535416,NaN
147130,727081,NaN
160774,2181839,NaN
215170,833913,NaN
228923,921872,NaN


In [21]:
region_data['Region'] = region_data['Region'].fillna('BUENOS AIRES')

In [22]:
region_data['Region'].isna().sum()

np.int64(0)

In [23]:
# Ya me asegure de que Region no tenga nulos
# Borro la columna region vacia
training_window.drop(['Region'], axis=1, inplace=True)

# Agrego la columna con los nuevos datos
training_window = training_window.merge(region_data[['client_id', 'Region']], on='client_id', how='left')
training_window

C:\Users\Ezequiel\AppData\Local\Temp\ipykernel_9212\1568007667.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_window.drop(['Region'], axis=1, inplace=True)


,client_id,Target,Month,First_product_dt,Last_product_dt,CreditCard_Premium,CreditCard_Active,CreditCard_CoBranding,Loan_Active,Mortgage_Active,...,CreditCard_Payment_External,CreditCard_Payment_Cash,CreditCard_Payment_Web,CreditCard_Payment_ATM,CreditCard_Payment_TAS,Investment_Numbers,Mobile,Email,CreditCard_Product,Region
0,5856970,1.0,2018-10-01,2013-10-23,2019-01-10,No,Yes,No,No,No,...,0.0,0.0,0.0,0.0,0.0,1.0,Yes,Yes,NaN,AMBA Resto
1,6371753,0.0,2018-09-01,2015-07-29,2018-06-02,No,No,No,No,No,...,0.0,0.0,0.0,0.0,0.0,0.0,Yes,No,NaN,REGION CENTRO
2,5928737,0.0,2019-01-01,2016-08-31,2018-12-27,No,No,No,No,No,...,0.0,0.0,0.0,0.0,0.0,0.0,Yes,Yes,NaN,REGION NORTE GRANDE ARGENTINO
3,475064,0.0,2018-12-01,2014-07-13,2017-11-30,No,Yes,No,No,No,...,0.0,0.0,0.0,0.0,0.0,0.0,Yes,Yes,NaN,REGION CUYO
4,3615172,0.0,2018-09-01,2017-12-27,2017-12-28,No,No,No,No,No,...,0.0,0.0,0.0,0.0,0.0,0.0,Yes,No,NaN,REGION CENTRO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139141,1673642,0.0,2018-11-01,2017-08-18,2017-09-26,No,Yes,No,No,No,...,0.0,0.0,1.0,0.0,0.0,0.0,No,Yes,NaN,BUENOS AIRES
139142,6145735,1.0,2018-11-01,2014-10-26,2014-10-26,No,Yes,No,No,No,...,0.0,0.0,1.0,0.0,0.0,0.0,Yes,Yes,NaN,REGION PATAGONICA
139143,5638786,1.0,2018-11-01,2012-12-26,2017-03-08,No,Yes,No,No,No,...,0.0,0.0,0.0,0.0,1.0,0.0,Yes,No,NaN,REGION NORTE GRANDE ARGENTINO
139144,3824781,0.0,2018-08-01,2014-11-27,2019-01-04,No,No,No,No,No,...,0.0,0.0,0.0,0.0,0.0,0.0,Yes,Yes,NaN,BUENOS AIRES


In [24]:
# 3) CreditCard_Product: Es la terecer columna con nulos
training_window['CreditCard_Product'].value_counts()

Series([], Name: count, dtype: int64)

In [25]:
data['CreditCard_Product'].value_counts()

CreditCard_Product
J55660104XX012    17298
J55660202XX012    12047
J55660124XX012     4978
J55660102XX012      834
J55660123XX012      774
J55660702XX012      567
J55661002XX012      138
Name: count, dtype: int64

In [26]:
data['CreditCard_Product'].shape

(238616,)

In [27]:
# Como todos los valores son nulos, procedo a intentar hacer lo mismo que con Region, pero teniendo en cuenta el pasado
# y el futuro
CreditCard_Product_data = data_final.loc[data_final['Month'].isin(['2019-03-01']), ['client_id', 'CreditCard_Product']]
CreditCard_Active_last_month = data_final.loc[data_final['Month'].isin(['2019-01-01']), ['client_id', 'CreditCard_Active']]
merged = CreditCard_Product_data.merge(CreditCard_Active_last_month,on='client_id',how='left')
merged

,client_id,CreditCard_Product,CreditCard_Active
0,2739521,J55660104XX012,Yes
1,5025967,NaN,No
2,2799736,NaN,No
3,6722691,J55660202XX012,Yes
4,5425606,J55660104XX012,Yes
...,...,...,...
23186,7328811,NaN,No
23187,6687431,NaN,No
23188,1899373,NaN,No
23189,6448833,NaN,No


In [28]:
# Me fijo si existen valores presentes CreditCard_Active = No y valores futuros CreditCard_Product != null, en ese caso
# los modifico para priorizar los datos del pasado
merged[(merged['CreditCard_Active'] == 'No') & (merged['CreditCard_Product'].notna())]

,client_id,CreditCard_Product,CreditCard_Active
68,3896692,J55660202XX012,No
111,59130,J55660104XX012,No
173,5651323,J55660202XX012,No
206,6315518,J55660104XX012,No
352,4623331,J55660124XX012,No
...,...,...,...
22628,3860521,J55660202XX012,No
22667,3254450,J55660104XX012,No
22681,4141,J55660202XX012,No
22820,6872951,J55660104XX012,No


In [29]:
# Veo que existen dichos datos, los modifico 
merged.loc[(merged['CreditCard_Active'] == 'No') & (merged['CreditCard_Product'].notna()),'CreditCard_Product'] = 'No tiene'
merged[(merged['CreditCard_Active'] == 'No') & (merged['CreditCard_Product'].notna())]

,client_id,CreditCard_Product,CreditCard_Active
68,3896692,No tiene,No
111,59130,No tiene,No
173,5651323,No tiene,No
206,6315518,No tiene,No
352,4623331,No tiene,No
...,...,...,...
22628,3860521,No tiene,No
22667,3254450,No tiene,No
22681,4141,No tiene,No
22820,6872951,No tiene,No


In [30]:
merged.drop(['CreditCard_Active'], axis=1, inplace=True)
# Veo que todavia tiene nulos, de 23mil clientes 8081 no tienen CreditCard_Product
merged['CreditCard_Product'].isna().sum()

np.int64(8081)

In [31]:
# A los que no tienen les reemplazo los nulos por "No tiene"
merged['CreditCard_Product'] = merged['CreditCard_Product'].fillna('No tiene')
merged['CreditCard_Product'].isna().sum()

np.int64(0)

In [32]:
# Borro la columna CreditCard_Product
training_window.drop(['CreditCard_Product'], axis=1, inplace=True)

# Agrego la columna con los nuevos datos
training_window = training_window.merge(merged[['client_id', 'CreditCard_Product']], on='client_id', how='left')
training_window

,client_id,Target,Month,First_product_dt,Last_product_dt,CreditCard_Premium,CreditCard_Active,CreditCard_CoBranding,Loan_Active,Mortgage_Active,...,CreditCard_Payment_External,CreditCard_Payment_Cash,CreditCard_Payment_Web,CreditCard_Payment_ATM,CreditCard_Payment_TAS,Investment_Numbers,Mobile,Email,Region,CreditCard_Product
0,5856970,1.0,2018-10-01,2013-10-23,2019-01-10,No,Yes,No,No,No,...,0.0,0.0,0.0,0.0,0.0,1.0,Yes,Yes,AMBA Resto,J55660202XX012
1,6371753,0.0,2018-09-01,2015-07-29,2018-06-02,No,No,No,No,No,...,0.0,0.0,0.0,0.0,0.0,0.0,Yes,No,REGION CENTRO,No tiene
2,5928737,0.0,2019-01-01,2016-08-31,2018-12-27,No,No,No,No,No,...,0.0,0.0,0.0,0.0,0.0,0.0,Yes,Yes,REGION NORTE GRANDE ARGENTINO,No tiene
3,475064,0.0,2018-12-01,2014-07-13,2017-11-30,No,Yes,No,No,No,...,0.0,0.0,0.0,0.0,0.0,0.0,Yes,Yes,REGION CUYO,J55660202XX012
4,3615172,0.0,2018-09-01,2017-12-27,2017-12-28,No,No,No,No,No,...,0.0,0.0,0.0,0.0,0.0,0.0,Yes,No,REGION CENTRO,No tiene
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139141,1673642,0.0,2018-11-01,2017-08-18,2017-09-26,No,Yes,No,No,No,...,0.0,0.0,1.0,0.0,0.0,0.0,No,Yes,BUENOS AIRES,J55660104XX012
139142,6145735,1.0,2018-11-01,2014-10-26,2014-10-26,No,Yes,No,No,No,...,0.0,0.0,1.0,0.0,0.0,0.0,Yes,Yes,REGION PATAGONICA,J55660202XX012
139143,5638786,1.0,2018-11-01,2012-12-26,2017-03-08,No,Yes,No,No,No,...,0.0,0.0,0.0,0.0,1.0,0.0,Yes,No,REGION NORTE GRANDE ARGENTINO,J55660202XX012
139144,3824781,0.0,2018-08-01,2014-11-27,2019-01-04,No,No,No,No,No,...,0.0,0.0,0.0,0.0,0.0,0.0,Yes,Yes,BUENOS AIRES,No tiene


In [33]:
# Me fijo que no queden valores nulos
training_window.columns[training_window.isnull().any()].tolist()
# Eso fue el tratamiento de valores nulos

[]

In [34]:
# Paso a tratar los Outliers

In [35]:
#Bines borrar
#numeric_columns = training_window[['SavingAccount_Balance_FirstDate', 'SavingAccount_Balance_LastDate', 'SavingAccount_Balance_Average', 
#    'SavingAccount_Days_with_use', 'SavingAccount_Days_with_Credits', 'SavingAccount_Days_with_Debits', 'SavingAccount_Salary_Payment_Transactions', 
#    'SavingAccount_Transfer_In_Transactions', 'SavingAccount_ATM_Extraction_Transactions', 'SavingAccount_Service_Payment_Transactions', 
#    'SavingAccount_CreditCard_Payment_Transactions', 'SavingAccount_Transfer_Out_Transactions', 'SavingAccount_DebitCard_Spend_Transactions', 
#    'SavingAccount_Transactions_Transactions', 'SavingAccount_Credits_Transactions', 'SavingAccount_Debits_Transactions', 
#    'SavingAccount_Salary_Payment_Amount', 'SavingAccount_Transfer_In_Amount', 'SavingAccount_ATM_Extraction_Amount', 
#    'SavingAccount_Service_Payment_Amount', 'SavingAccount_CreditCard_Payment_Amount', 'SavingAccount_Transfer_Out_Amount', 
#    'SavingAccount_DebitCard_Spend_Amount', 'SavingAccount_Total_Amount', 'SavingAccount_Credits_Amounts', 'SavingAccount_Debits_Amounts', 
#    'Operations_Bank', 'Operations_Terminal', 'Operations_HomeBanking', 'Operations_Mobile', 'Operations_Ivr', 'Operations_Telemarketer', 
#    'Operations_ATM', 'CreditCard_Balance_ARG', 'CreditCard_Balance_DOLLAR', 'CreditCard_Total_Limit', 'CreditCard_Total_Spending', 
#    'CreditCard_Spending_1_Installment', 'CreditCard_Spending_Installments', 'CreditCard_Spending_CrossBoarder', 'CreditCard_Spending_Aut_Debits', 
#    'CreditCard_Revolving', 'CreditCard_Payment_Aut_Debit', 'CreditCard_Payment_External', 'CreditCard_Payment_Cash', 'CreditCard_Payment_Web', 
#    'CreditCard_Payment_ATM', 'CreditCard_Payment_TAS', 'Investment_Numbers']]
#
#for col in numeric_columns:
#    q2 = training_window[col].quantile(0.75)
#    q3 = training_window[col].quantile(0.90)
#
#    new_col = f"{col}_bin"
#
#    training_window[new_col] = np.where(
#        training_window[col] == 0, 0,
#        np.where(training_window[col] < q2, 2,
#            np.where(training_window[col] < q3, 3, 4)
#        )
#    )
#
#    print(f"{new_col} value counts:")
#    print(training_window[new_col].value_counts(), "\n")

In [36]:
training_window.shape

(139146, 77)

In [37]:
# Eso fue el tratamiento de outliers
# Ahora paso a trabajar con las identity features

In [38]:
data_if = training_window[training_window['Month'] == '2019-01-01'][['client_id', 'Mobile', 'Email', 'CreditCard_Premium', 'CreditCard_Active', 'CreditCard_CoBranding', 'Loan_Active', 
    'Mortgage_Active', 'SavingAccount_Active_ARG_Salary', 'SavingAccount_Active_ARG', 'SavingAccount_Active_DOLLAR', 'DebitCard_Active', 
    'Investment_Active', 'Package_Active', 'Insurance_Life', 'Insurance_Home', 'Insurance_Accidents', 'Insurance_Mobile', 'Insurance_ATM', 
    'Insurance_Unemployment', 'Sex', 'Client_Age_grp', 'Region', 'CreditCard_Product']]
data_if

,client_id,Mobile,Email,CreditCard_Premium,CreditCard_Active,CreditCard_CoBranding,Loan_Active,Mortgage_Active,SavingAccount_Active_ARG_Salary,SavingAccount_Active_ARG,...,Insurance_Life,Insurance_Home,Insurance_Accidents,Insurance_Mobile,Insurance_ATM,Insurance_Unemployment,Sex,Client_Age_grp,Region,CreditCard_Product
2,5928737,Yes,Yes,No,No,No,No,No,No,Yes,...,No,No,No,No,No,No,M,Entre 30 y 39 años,REGION NORTE GRANDE ARGENTINO,No tiene
8,6018047,Yes,Yes,Yes,Yes,No,No,No,No,Yes,...,No,No,No,No,No,No,M,Entre 60 y 64 años,BUENOS AIRES,J55660104XX012
9,5359038,Yes,Yes,Yes,Yes,No,Yes,No,Yes,Yes,...,No,No,No,No,No,No,M,Entre 40 y 49 años,REGION NORTE GRANDE ARGENTINO,J55660104XX012
11,6890812,Yes,Yes,No,No,No,Yes,No,No,Yes,...,No,No,No,No,No,No,M,Entre 40 y 49 años,REGION NORTE GRANDE ARGENTINO,No tiene
13,115383,No,No,No,No,No,No,No,No,Yes,...,No,No,No,No,No,No,M,Mayor a 70 años,AMBA Resto,No tiene
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139131,6570413,Yes,Yes,No,Yes,No,No,No,No,Yes,...,No,No,No,No,No,No,F,Entre 30 y 39 años,REGION PATAGONICA,J55660104XX012
139132,6258895,Yes,Yes,No,No,No,No,No,No,Yes,...,No,No,No,No,No,No,F,Entre 30 y 39 años,BUENOS AIRES,No tiene
139138,6397274,No,Yes,No,Yes,No,No,No,No,No,...,No,No,No,No,No,No,M,Entre 40 y 49 años,REGION CENTRO,J55660104XX012
139139,6007291,No,No,Yes,Yes,No,No,No,No,No,...,No,No,No,No,No,No,F,Entre 40 y 49 años,CABA Centro/Norte,J55660202XX012


In [39]:
# Primero las binarias
pd.set_option('future.no_silent_downcasting', True)
data_if = data_if.replace({'Yes': 1, 'No': 0})
# esta funcion hay que aplicarla con cuidado, reemplaza y castea todos los yes | no que encuentre
pd.set_option('future.no_silent_downcasting', True)
data_if = data_if.replace({'M': 1, 'F': 0})
# esta funcion hay que aplicarla con cuidado, reemplaza y castea todos los M | F que encuentre

In [40]:
data_if

,client_id,Mobile,Email,CreditCard_Premium,CreditCard_Active,CreditCard_CoBranding,Loan_Active,Mortgage_Active,SavingAccount_Active_ARG_Salary,SavingAccount_Active_ARG,...,Insurance_Life,Insurance_Home,Insurance_Accidents,Insurance_Mobile,Insurance_ATM,Insurance_Unemployment,Sex,Client_Age_grp,Region,CreditCard_Product
2,5928737,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,Entre 30 y 39 años,REGION NORTE GRANDE ARGENTINO,No tiene
8,6018047,1,1,1,1,0,0,0,0,1,...,0,0,0,0,0,0,1,Entre 60 y 64 años,BUENOS AIRES,J55660104XX012
9,5359038,1,1,1,1,0,1,0,1,1,...,0,0,0,0,0,0,1,Entre 40 y 49 años,REGION NORTE GRANDE ARGENTINO,J55660104XX012
11,6890812,1,1,0,0,0,1,0,0,1,...,0,0,0,0,0,0,1,Entre 40 y 49 años,REGION NORTE GRANDE ARGENTINO,No tiene
13,115383,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,Mayor a 70 años,AMBA Resto,No tiene
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139131,6570413,1,1,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,Entre 30 y 39 años,REGION PATAGONICA,J55660104XX012
139132,6258895,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,Entre 30 y 39 años,BUENOS AIRES,No tiene
139138,6397274,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,Entre 40 y 49 años,REGION CENTRO,J55660104XX012
139139,6007291,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,Entre 40 y 49 años,CABA Centro/Norte,J55660202XX012


In [41]:
# Eso fue el tratamiento de las identity features
# Ahora sigo con las categoricas
data_if['Client_Age_grp'].value_counts()

Client_Age_grp
Entre 40 y 49 años    6563
Entre 30 y 39 años    5308
Entre 50 y 59 años    5149
Entre 60 y 64 años    2122
Entre 65 y 69 años    1745
Mayor a 70 años       1315
Entre 18 y 29 años     989
Name: count, dtype: int64

In [42]:
data_buffer_categorica = data_if.merge(training_window[['client_id', 'Target']],on='client_id',how='left')
data_buffer_categorica

,client_id,Mobile,Email,CreditCard_Premium,CreditCard_Active,CreditCard_CoBranding,Loan_Active,Mortgage_Active,SavingAccount_Active_ARG_Salary,SavingAccount_Active_ARG,...,Insurance_Home,Insurance_Accidents,Insurance_Mobile,Insurance_ATM,Insurance_Unemployment,Sex,Client_Age_grp,Region,CreditCard_Product,Target
0,5928737,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,Entre 30 y 39 años,REGION NORTE GRANDE ARGENTINO,No tiene,0.0
1,5928737,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,Entre 30 y 39 años,REGION NORTE GRANDE ARGENTINO,No tiene,0.0
2,5928737,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,Entre 30 y 39 años,REGION NORTE GRANDE ARGENTINO,No tiene,0.0
3,5928737,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,Entre 30 y 39 años,REGION NORTE GRANDE ARGENTINO,No tiene,0.0
4,5928737,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,Entre 30 y 39 años,REGION NORTE GRANDE ARGENTINO,No tiene,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139141,6412619,1,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,Entre 60 y 64 años,REGION PATAGONICA,No tiene,0.0
139142,6412619,1,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,Entre 60 y 64 años,REGION PATAGONICA,No tiene,0.0
139143,6412619,1,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,Entre 60 y 64 años,REGION PATAGONICA,No tiene,0.0
139144,6412619,1,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,Entre 60 y 64 años,REGION PATAGONICA,No tiene,0.0


In [43]:
# Tengo que trabajar con Client_Age_grp, Region, CreditCard_Product
# 1) Empiezo por Client_Age_grp
data_buffer_categorica[['Client_Age_grp', 'Target']].value_counts().reset_index()

,Client_Age_grp,Target,count
0,Entre 40 y 49 años,0.0,29418
1,Entre 30 y 39 años,0.0,24564
2,Entre 50 y 59 años,0.0,20118
3,Entre 50 y 59 años,1.0,10776
4,Entre 40 y 49 años,1.0,9960
5,Entre 60 y 64 años,0.0,8076
6,Entre 30 y 39 años,1.0,7284
7,Entre 65 y 69 años,0.0,6306
8,Mayor a 70 años,0.0,5286
9,Entre 60 y 64 años,1.0,4656


In [44]:
# Porcentaje que representa cuantos son target de cada rango
di = {
    "Entre 18 y 29 años": 25,
    "Entre 30 y 39 años": 23,
    "Entre 40 y 49 años": 25,
    "Entre 50 y 59 años": 35,
    "Entre 60 y 64 años": 37,
    "Entre 65 y 69 años": 40,
    "Mayor a 70 años": 33
     }

# Agrego la columna generada a data_if
data_if['Client_Age_grp_Target'] = data_if['Client_Age_grp'].map(di)

data_if[['Client_Age_grp', 'Client_Age_grp_Target']].value_counts()

Client_Age_grp      Client_Age_grp_Target
Entre 40 y 49 años  25                       6563
Entre 30 y 39 años  23                       5308
Entre 50 y 59 años  35                       5149
Entre 60 y 64 años  37                       2122
Entre 65 y 69 años  40                       1745
Mayor a 70 años     33                       1315
Entre 18 y 29 años  25                        989
Name: count, dtype: int64

In [45]:
# Borro la columna que no necesito mas
data_if.drop(['Client_Age_grp'], axis=1, inplace=True)

In [46]:
data_if

,client_id,Mobile,Email,CreditCard_Premium,CreditCard_Active,CreditCard_CoBranding,Loan_Active,Mortgage_Active,SavingAccount_Active_ARG_Salary,SavingAccount_Active_ARG,...,Insurance_Life,Insurance_Home,Insurance_Accidents,Insurance_Mobile,Insurance_ATM,Insurance_Unemployment,Sex,Region,CreditCard_Product,Client_Age_grp_Target
2,5928737,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,REGION NORTE GRANDE ARGENTINO,No tiene,23
8,6018047,1,1,1,1,0,0,0,0,1,...,0,0,0,0,0,0,1,BUENOS AIRES,J55660104XX012,37
9,5359038,1,1,1,1,0,1,0,1,1,...,0,0,0,0,0,0,1,REGION NORTE GRANDE ARGENTINO,J55660104XX012,25
11,6890812,1,1,0,0,0,1,0,0,1,...,0,0,0,0,0,0,1,REGION NORTE GRANDE ARGENTINO,No tiene,25
13,115383,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,AMBA Resto,No tiene,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139131,6570413,1,1,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,REGION PATAGONICA,J55660104XX012,23
139132,6258895,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,BUENOS AIRES,No tiene,23
139138,6397274,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,REGION CENTRO,J55660104XX012,25
139139,6007291,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,CABA Centro/Norte,J55660202XX012,25


In [47]:
# 2) Sigo con Region
data_buffer_categorica[['Region', 'Target']].value_counts().reset_index()

,Region,Target,count
0,BUENOS AIRES,0.0,29112
1,REGION CENTRO,0.0,20778
2,REGION NORTE GRANDE ARGENTINO,0.0,15708
3,BUENOS AIRES,1.0,12828
4,REGION PATAGONICA,0.0,9672
5,CABA Centro/Norte,0.0,8616
6,AMBA Resto,0.0,7416
7,REGION CUYO,0.0,6906
8,REGION CENTRO,1.0,6696
9,REGION NORTE GRANDE ARGENTINO,1.0,6336


In [48]:
# Porcentaje que representa cuantos son target de cada rango
dr = {
    "BUENOS AIRES": 31,
    "REGION CENTRO": 24,
    "REGION NORTE GRANDE ARGENTINO": 29,
    "REGION PATAGONICA": 34,
    "CABA Centro/Norte": 30,
    "AMBA Resto": 32,
    "REGION CUYO": 30
     }

# Agrego la columna generada a data_if
data_if['Region_Target'] = data_if['Region'].map(dr)

data_if[['Region', 'Region_Target']].value_counts()

Region                         Region_Target
BUENOS AIRES                   31               6990
REGION CENTRO                  24               4579
REGION NORTE GRANDE ARGENTINO  29               3674
REGION PATAGONICA              34               2432
CABA Centro/Norte              30               2048
AMBA Resto                     32               1828
REGION CUYO                    30               1640
Name: count, dtype: int64

In [49]:
# Borro la columna que no necesito mas
data_if.drop(['Region'], axis=1, inplace=True)

In [50]:
data_if

,client_id,Mobile,Email,CreditCard_Premium,CreditCard_Active,CreditCard_CoBranding,Loan_Active,Mortgage_Active,SavingAccount_Active_ARG_Salary,SavingAccount_Active_ARG,...,Insurance_Life,Insurance_Home,Insurance_Accidents,Insurance_Mobile,Insurance_ATM,Insurance_Unemployment,Sex,CreditCard_Product,Client_Age_grp_Target,Region_Target
2,5928737,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,No tiene,23,29
8,6018047,1,1,1,1,0,0,0,0,1,...,0,0,0,0,0,0,1,J55660104XX012,37,31
9,5359038,1,1,1,1,0,1,0,1,1,...,0,0,0,0,0,0,1,J55660104XX012,25,29
11,6890812,1,1,0,0,0,1,0,0,1,...,0,0,0,0,0,0,1,No tiene,25,29
13,115383,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,No tiene,33,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139131,6570413,1,1,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,J55660104XX012,23,34
139132,6258895,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,No tiene,23,31
139138,6397274,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,J55660104XX012,25,24
139139,6007291,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,J55660202XX012,25,30


In [51]:
# 2) Sigo con CreditCard_Product
data_buffer_categorica[['CreditCard_Product', 'Target']].value_counts().reset_index()

,CreditCard_Product,Target,count
0,No tiene,0.0,47286
1,J55660104XX012,0.0,26370
2,J55660104XX012,1.0,23304
3,J55660202XX012,0.0,21858
4,J55660202XX012,1.0,12804
5,No tiene,1.0,3096
6,J55660102XX012,0.0,1584
7,J55660702XX012,0.0,864
8,J55660102XX012,1.0,828
9,J55660702XX012,1.0,624


In [52]:
# Porcentaje que representa cuantos son target de cada rango
dp = {
    "J55660102XX012": 34,
    "J55660104XX012": 47,
    "J55660123XX012": 0,
    "J55660124XX012": 38,
    "J55660202XX012": 37,
    "J55660702XX012": 42,
    "J55661002XX012": 62,
    "No tiene": 6
     }

# Agrego la columna generada a data_if
data_if['CreditCard_Product_Target'] = data_if['CreditCard_Product'].map(dp)

data_if[['CreditCard_Product', 'CreditCard_Product_Target']].value_counts()

CreditCard_Product  CreditCard_Product_Target
No tiene            6                            8397
J55660104XX012      47                           8279
J55660202XX012      37                           5777
J55660102XX012      34                            402
J55660702XX012      42                            248
J55661002XX012      62                             58
J55660124XX012      38                             29
J55660123XX012      0                               1
Name: count, dtype: int64

In [53]:
# Borro la columna que no necesito mas
data_if.drop(['CreditCard_Product'], axis=1, inplace=True)

In [54]:
data_if

,client_id,Mobile,Email,CreditCard_Premium,CreditCard_Active,CreditCard_CoBranding,Loan_Active,Mortgage_Active,SavingAccount_Active_ARG_Salary,SavingAccount_Active_ARG,...,Insurance_Life,Insurance_Home,Insurance_Accidents,Insurance_Mobile,Insurance_ATM,Insurance_Unemployment,Sex,Client_Age_grp_Target,Region_Target,CreditCard_Product_Target
2,5928737,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,23,29,6
8,6018047,1,1,1,1,0,0,0,0,1,...,0,0,0,0,0,0,1,37,31,47
9,5359038,1,1,1,1,0,1,0,1,1,...,0,0,0,0,0,0,1,25,29,47
11,6890812,1,1,0,0,0,1,0,0,1,...,0,0,0,0,0,0,1,25,29,6
13,115383,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,33,32,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139131,6570413,1,1,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,23,34,47
139132,6258895,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,23,31,6
139138,6397274,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,25,24,47
139139,6007291,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,25,30,37


In [55]:
# Ahora paso a trabajar las transform feature, las puedo tomar de los meses que quiera
# Agrego las vistas en clase
data_tf = training_window[training_window['Month'] == '2019-01-01'][['client_id']]

# Cantidad de operaciones totales por cliente
data_tf['Operation_total'] = training_window['Operations_Bank'] + training_window['Operations_Terminal'] + training_window[ 'Operations_HomeBanking'] + training_window[
       'Operations_Mobile'] + training_window['Operations_Ivr'] + training_window['Operations_Telemarketer'] + training_window['Operations_ATM']
data_tf

,client_id,Operation_total
2,5928737,0.0
8,6018047,1.0
9,5359038,16.0
11,6890812,5.0
13,115383,0.0
...,...,...
139131,6570413,6.0
139132,6258895,0.0
139138,6397274,0.0
139139,6007291,0.0


In [58]:
# Cantidad de operaciones digitales por cliente
data_tf['Operation_digitales'] = training_window['Operations_HomeBanking'] + training_window['Operations_Mobile']
+ training_window['Operations_Ivr'] + training_window['Operations_Telemarketer']
data_tf

,client_id,Operation_total,Operation_digitales
2,5928737,0.0,0.0
8,6018047,1.0,0.0
9,5359038,16.0,16.0
11,6890812,5.0,5.0
13,115383,0.0,0.0
...,...,...,...
139131,6570413,6.0,3.0
139132,6258895,0.0,0.0
139138,6397274,0.0,0.0
139139,6007291,0.0,0.0


In [59]:
# Porcentaje de operaciones que fueron digitales por cliente
data_tf['Operation_digitales_porc'] = np.where(data_tf['Operation_total'] != 0, data_tf['Operation_digitales'] / data_tf['Operation_total'], 0)
data_tf

,client_id,Operation_total,Operation_digitales,Operation_digitales_porc
2,5928737,0.0,0.0,0.0
8,6018047,1.0,0.0,0.0
9,5359038,16.0,16.0,1.0
11,6890812,5.0,5.0,1.0
13,115383,0.0,0.0,0.0
...,...,...,...,...
139131,6570413,6.0,3.0,0.5
139132,6258895,0.0,0.0,0.0
139138,6397274,0.0,0.0,0.0
139139,6007291,0.0,0.0,0.0


In [78]:
# Porcentaje de operaciones digitales que fueron por home banking
data_tf['Operation_digitales_hb_porc'] = training_window['Operations_HomeBanking'] / data_tf['Operation_digitales']
data_tf['Operation_digitales_hb_porc'] = data_tf['Operation_digitales_hb_porc'].fillna(0)
data_tf['Operation_digitales_hb_porc'].value_counts()

Operation_digitales_hb_porc
0.000000    16415
1.000000     5042
0.500000      240
0.333333      114
0.666667      105
            ...  
0.484848        1
0.448276        1
0.774194        1
0.323529        1
0.447368        1
Name: count, Length: 209, dtype: int64

In [77]:
# Porcentaje de operaciones que fueron por ATM
data_tf['Operations_ATM_porc'] = training_window['Operations_ATM'] / data_tf['Operation_total']
data_tf['Operations_ATM_porc'] = data_tf['Operations_ATM_porc'].fillna(0)
data_tf['Operations_ATM_porc'] = data_tf['Operations_ATM_porc'].replace([np.inf, -np.inf], 0)
data_tf['Operations_ATM_porc'].value_counts()

Operations_ATM_porc
0.000000    21677
1.000000      223
0.500000      132
0.333333       96
0.250000       66
            ...  
0.031250        1
0.017857        1
0.533333        1
0.277778        1
0.025641        1
Name: count, Length: 124, dtype: int64

In [76]:
# Porcentaje de operaciones que fueron por ventanilla
data_tf['Operations_Bank_porc'] = training_window['Operations_Bank'] / data_tf['Operation_total']
data_tf['Operations_Bank_porc'] = data_tf['Operations_Bank_porc'].fillna(0)
data_tf['Operations_Bank_porc'] = data_tf['Operations_Bank_porc'].replace([np.inf, -np.inf], 0)
data_tf['Operations_Bank_porc'].value_counts()

Operations_Bank_porc
0.000000    20932
1.000000      607
0.500000      406
0.333333      199
0.250000      126
            ...  
0.414634        1
0.156250        1
0.160000        1
0.120000        1
0.461538        1
Name: count, Length: 87, dtype: int64

In [80]:
# Porcentaje de dinero gastado en relacion al limite con la tarjeta
#data_tf['Credit_Card_Spending_limit_porc'] = training_window['CreditCard_Total_Limit']
training_window[training_window['Month'] == '2019-01-01'][['CreditCard_Total_Limit']]

,CreditCard_Total_Limit
2,0.0
8,80000.0
9,64000.0
11,0.0
13,0.0
...,...
139131,28000.0
139132,0.0
139138,40000.0
139139,96000.0


In [ ]:
# Sigo con las agregadas

In [ ]:
var = ['SavingAccount_Active_ARG_Salary','SavingAccount_Active_ARG','SavingAccount_Active_DOLLAR', 'SavingAccount_Balance_FirstDate', 
       'SavingAccount_Balance_LastDate', 'SavingAccount_Balance_Average', 'SavingAccount_Days_with_use', 
       'SavingAccount_Days_with_Credits', 'SavingAccount_Days_with_Debits', 'SavingAccount_Salary_Payment_Transactions', 
       'SavingAccount_Transfer_In_Transactions', 'SavingAccount_ATM_Extraction_Transactions', 'SavingAccount_Service_Payment_Transactions', 
       'SavingAccount_CreditCard_Payment_Transactions', 'SavingAccount_Transfer_Out_Transactions', 'SavingAccount_DebitCard_Spend_Transactions', 
       'SavingAccount_Transactions_Transactions', 'SavingAccount_Credits_Transactions', 'SavingAccount_Debits_Transactions', 
       'SavingAccount_Salary_Payment_Amount', 'SavingAccount_Transfer_In_Amount', 'SavingAccount_ATM_Extraction_Amount', 
       'SavingAccount_Service_Payment_Amount', 'SavingAccount_CreditCard_Payment_Amount', 'SavingAccount_Transfer_Out_Amount', 
       'SavingAccount_DebitCard_Spend_Amount', 'SavingAccount_Total_Amount', 'SavingAccount_Credits_Amounts', 'SavingAccount_Debits_Amounts', 
       'Operations_Bank', 'Operations_Terminal', 'Operations_HomeBanking', 'Operations_Mobile', 'Operations_Ivr', 'Operations_Telemarketer', 
       'Operations_ATM', 'CreditCard_Balance_ARG', 'CreditCard_Balance_DOLLAR', 'CreditCard_Total_Limit', 'CreditCard_Total_Spending', 
       'CreditCard_Spending_1_Installment', 'CreditCard_Spending_Installments', 'CreditCard_Spending_CrossBoarder', 'CreditCard_Spending_Aut_Debits', 
       'CreditCard_Revolving', 'CreditCard_Payment_Aut_Debit', 'CreditCard_Payment_External', 'CreditCard_Payment_Cash', 'CreditCard_Payment_Web', 
       'CreditCard_Payment_ATM', 'CreditCard_Payment_TAS', 'Investment_Numbers']

print(len(var))


In [ ]:
columns_a_reemplazar = ['SavingAccount_Active_ARG_Salary','SavingAccount_Active_ARG','SavingAccount_Active_DOLLAR']

map_id ={
    'Yes':1,
    'No':0,
    False:0,
    True :1
}

for column in columns_a_reemplazar:
    try:
        training_window[column] = training_window[column].replace(map_id)
        training_window[column] = training_window[column].fillna(0)
        print(paso)
    except:
        pass

In [ ]:
data_final_sa_abt = training_window.groupby(['client_id'])[var].agg(['sum', 'max','min', 'mean', 'median', np.count_nonzero, 'nunique']).reset_index()
data_final_sa_abt.columns

In [ ]:
len(data_final_sa_abt.columns)

In [ ]:
data_final_sa_abt.columns = ['_'.join(x) for x in np.array(data_final_sa_abt.columns)]
data_final_sa_abt.columns

In [ ]:
# diferencia entre meses

# mes 5
data_final_sa_abt_mes_5 = training_window[training_window['Month'] == '2018-08-01']

# mes 0
data_final_sa_abt_mes0 = training_window[training_window['Month'] == '2019-01-01']

#join
data_final_sa_mes_5_y_0 = data_final_sa_abt_mes0.merge(data_final_sa_abt_mes_5, how='inner', on='client_id')
data_final_sa_mes_5_y_0.columns.tolist()

In [ ]:
data_final_sa_mes_5_y_0

In [ ]:
for x in var:
    try:
        print(x)
        data_final_sa_mes_5_y_0[x + '_diff_mes_5_y_0_abs'] = data_final_sa_mes_5_y_0[x + '_x'] -  data_final_sa_mes_5_y_0[x + '_y']    
        data_final_sa_mes_5_y_0[x + '_diff_mes_5_y_0_rel'] = data_final_sa_mes_5_y_0[x + '_x'] /  data_final_sa_mes_5_y_0[x + '_y']
        data_final_sa_mes_5_y_0[x + '_diff_mes_5_y_0_rel2'] = 1 - (data_final_sa_mes_5_y_0[x + '_x'] /  data_final_sa_mes_5_y_0[x + '_y'])
    except:
        pass

In [ ]:
var = [x for x in data_final_sa_mes_5_y_0.columns if x.find('_diff_mes_5_y_0') > 0]
var

In [ ]:
var += ['client_id']
data_final_sa_mes_5_y_0_final = data_final_sa_mes_5_y_0[var]

In [ ]:
# ABT FINAL

In [ ]:
data_final_sa_abt['client_id'] = data_final_sa_abt['client_id_']
data_final_sa_abt.columns

In [ ]:
print(len(data_final_sa_abt))

In [ ]:
ABT = data_TGT.merge(data_if, how='inner', on='client_id')\
.merge(data_tf, how='inner', on='client_id')\
.merge(data_final_sa_abt, how='inner', on='client_id')

ABT.shape

In [ ]:
data_final_sa_abt.columns.tolist()